In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
!pip install bayesian-optimization

In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten
import tensorflow as tf
import numpy as np
import os
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder

from bayes_opt import BayesianOptimization

In [4]:
train=pd.read_csv("./train_data.csv", encoding='cp949')
test=pd.read_csv("./test_data.csv", encoding='cp949')

## 전처리

In [5]:
#결측치 확인
def shownull(df):
    global missval_name
    print(f"데이터셋 차원 확인 \n {df.shape}")
    missval = df.isnull().sum()[df.isnull().sum()!=0]
    print(f"결측값 있는 변수 확인 \n {missval}")
    missval_name = missval.index # 결측치가 존재하는 열이 저장된 변수

#이상치 확인
def zscore_out(df, threshold = 3):
    mean = np.mean(df)
    std = np.std(df)
    z_score = np.abs((df - df.mean()) / df.std())
    outliers = np.where(z_score > threshold)
    # outlier_idx = outliers.nonzero()[0]
    print(outliers[0])

#시드고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(84)

### 결측치 대체

In [9]:
# Train set에 대해서 시간 변수 결측치 대체

train['regdate'] = pd.to_datetime(train['regdate'])

# 시작일과 종료일 설정
start_date = train['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = train['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(train)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(train.loc[i, 'regdate']):
        train.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

#제대로 작동됨

In [10]:
# Test set에 대해서 시간 변수 결측치 대체

test['regdate'] = pd.to_datetime(test['regdate'])

# 시작일과 종료일 설정
start_date = test['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = test['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(test)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(test.loc[i, 'regdate']):
        test.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

### 독립변수 추가

In [11]:
# TRAIN 요일 변수 추가
daysofweek = []
for i in range(len(train)) :
    dayweek = train.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

train['dayofweek'] = daysofweek



#월요일 0 ~ 일요일6

<!--  -->

In [12]:
# TEST 요일 변수 추가

daysofweek = []
for i in range(len(test)) :
    dayweek = test.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

test['dayofweek'] = daysofweek

In [13]:
# TRAIN 시간 -> 10진법 변환
train['time'] = train['regdate'].dt.strftime('%H:%M').str.split().str[0]
train['time'] = train['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [14]:
# TEST 시간 -> 10진법 변환
test['time'] = test['regdate'].dt.strftime('%H:%M').str.split().str[0]
test['time'] = test['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [15]:
it_is_not_na = np.where(~train[['PIR']].isnull())[0]

train.loc[it_is_not_na,['PIR']] = train.loc[it_is_not_na,['PIR']].astype(np.int64)

### 결측치 대체

In [16]:
def make_area(df) :
    # 입력된 날짜 변수의 차이 계산
    date_range = df['regdate'].max() - df['regdate'].min()

    # 간격당 행의 개수 계산
    lowsbydate = ((date_range.days + 1) * 1440) #하루 1440분
    group_num = len(train) / lowsbydate

    for i in range(int(group_num)):
        start_index = i * lowsbydate
        end_index = (i + 1) * lowsbydate
        df.loc[start_index : end_index, 'area'] = i + 1

def inter_by_area(df):
    df.update(df.groupby('area').transform(lambda x: x.interpolate(method = 'pad')))
    return df

In [ ]:
train_nona = train.copy()
# cat_num_na(train_nona)
make_area(train_nona)
inter_by_area(train_nona)

test_nona = test.copy()
# cat_num_na(test_nona)
make_area(test_nona)
inter_by_area(test_nona)

### 파일 분할 및 스케일링

In [21]:
X_train = train_nona.drop('재실인원', axis=1)
X_train = X_train.drop(['regdate','area','dayofweek'], axis=1)
X_train= X_train.values
y_train = train_nona['재실인원'].values


X_test = test_nona.drop('재실인원', axis=1)
X_test = X_test.drop(['regdate','area','dayofweek'], axis=1)
X_test=X_test.values
y_test = test_nona['재실인원'].values

In [22]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

## timestep 지정을 위한 X값 shape 조정

In [23]:
timestep = 1
X_train_scaled = []
y_train_scaled = []
for i in range(len(train_nona) - timestep + 1):
    X_train_scaled.append(X_train[i:i+timestep])
    y_train_scaled.append(y_train[i+timestep-1])
X_train = np.array(X_train_scaled)
y_train = np.array(y_train_scaled)

In [24]:
X_test_scaled = []
y_test_scaled = []
for i in range(len(test_nona) - timestep + 1):
    X_test_scaled.append(X_test[i:i+timestep])
    y_test_scaled.append(y_test[i+timestep-1])
X_test = np.array(X_test_scaled)
y_test = np.array(y_test_scaled)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [26]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(311040, 1, 11) (77760, 1, 11) (311040,) (77760,)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,LSTM
from keras.layers import Input, Dense, Activation, Flatten, Dropout

In [28]:
mse =[]
def buildModel(neurons,drop,nb_batch,hidden_layers):  
    nb_epoch=5
    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = tf.keras.Input(shape=(timestep,11))
    x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=drop, activation='relu')(inputs)  
    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = tf.keras.layers.GRU(neurons * (2 ** (hidden_layers + 1)), return_sequences=True, recurrent_dropout=drop)(x)
    # output
    predictions = Dense(1)(x)


    # modeling
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



    # learning
    model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


    # evaluate the model with test data
    loss, mse =model.evaluate(X_test, y_test)
    
    if mse < 3.0 : 
        model.save(f"./model/gru/{mse}_gru.h5")
        print("모델이 저장 되었습니다.")
        print("mse : ",mse)
    
    return mse

In [29]:
pbounds = {
        # define the range of each parameter
        'drop' : (0.5,0.9),
        "neurons" : (12,24),
        'nb_batch' : (512,1024),
        'hidden_layers' : (0,3)}

In [30]:
bo=BayesianOptimization(f=buildModel, pbounds=pbounds, verbose=2, random_state=42 )

In [31]:
bo.maximize(
   init_points=2,
   n_iter= 100
)

|   iter    |  target   |   drop    | hidden... | nb_batch  |  neurons  |
-------------------------------------------------------------------------
Epoch 1/5
352/352 [==============================] - 24s 55ms/step - loss: 2.1258 - mean_squared_error: 2.1258 - val_loss: 2.0602 - val_mean_squared_error: 2.0602
Epoch 2/5
352/352 [==============================] - 19s 53ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0576 - val_mean_squared_error: 2.0576
Epoch 3/5
352/352 [==============================] - 19s 53ms/step - loss: 2.0753 - mean_squared_error: 2.0753 - val_loss: 2.0560 - val_mean_squared_error: 2.0560
Epoch 4/5
352/352 [==============================] - 19s 53ms/step - loss: 2.0741 - mean_squared_error: 2.0741 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 5/5
2657/2657 [==============================] - 22s 8ms/step - loss: 2.9016 - mean_squared_error: 2.9016
모델이 저장 되었습니다.
mse :  2.9016456604003906
| 1         | 2.902     | 0.6498    | 2.852     

Epoch 1/5
420/420 [==============================] - 12s 24ms/step - loss: 2.3116 - mean_squared_error: 2.3116 - val_loss: 2.0697 - val_mean_squared_error: 2.0697
Epoch 2/5
420/420 [==============================] - 10s 23ms/step - loss: 2.0786 - mean_squared_error: 2.0785 - val_loss: 2.0574 - val_mean_squared_error: 2.0574
Epoch 3/5
420/420 [==============================] - 10s 24ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 4/5
420/420 [==============================] - 10s 24ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 2.0561 - val_mean_squared_error: 2.0561
Epoch 5/5
2657/2657 [==============================] - 14s 5ms/step - loss: 72867.8125 - mean_squared_error: 72867.8125
| 7         | 7.287e+04 | 0.5332    | 0.3124    | 742.8     | 16.61     |
Epoch 1/5
417/417 [==============================] - 38s 76ms/step - loss: 2.1073 - mean_squared_error: 2.1073 - val_loss: 2.0561 - val_mean_squared_error

Epoch 1/5
417/417 [==============================] - 11s 23ms/step - loss: 2.2878 - mean_squared_error: 2.2878 - val_loss: 2.0755 - val_mean_squared_error: 2.0755
Epoch 2/5
417/417 [==============================] - 9s 22ms/step - loss: 2.0792 - mean_squared_error: 2.0792 - val_loss: 2.0568 - val_mean_squared_error: 2.0568
Epoch 3/5
417/417 [==============================] - 9s 23ms/step - loss: 2.0738 - mean_squared_error: 2.0738 - val_loss: 2.0578 - val_mean_squared_error: 2.0578
Epoch 4/5
417/417 [==============================] - 10s 23ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 2.0560 - val_mean_squared_error: 2.0560
Epoch 5/5
2657/2657 [==============================] - 13s 5ms/step - loss: 67544.3594 - mean_squared_error: 67544.3594
| 13        | 6.754e+04 | 0.6995    | 0.408     | 746.5     | 16.26     |
Epoch 1/5
418/418 [==============================] - 12s 24ms/step - loss: 2.3699 - mean_squared_error: 2.3699 - val_loss: 2.0758 - val_mean_squared_error: 

Epoch 3/5
583/583 [==============================] - 35s 59ms/step - loss: 2.0748 - mean_squared_error: 2.0748 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 4/5
583/583 [==============================] - 35s 59ms/step - loss: 2.0744 - mean_squared_error: 2.0744 - val_loss: 2.0567 - val_mean_squared_error: 2.0567
Epoch 5/5
2657/2657 [==============================] - 21s 8ms/step - loss: 2.3752 - mean_squared_error: 2.3752
모델이 저장 되었습니다.
mse :  2.3751931190490723
| 19        | 2.375     | 0.7856    | 2.243     | 534.2     | 16.83     |
Epoch 1/5
470/470 [==============================] - 30s 54ms/step - loss: 2.1157 - mean_squared_error: 2.1157 - val_loss: 2.0605 - val_mean_squared_error: 2.0605
Epoch 2/5
470/470 [==============================] - 25s 52ms/step - loss: 2.0755 - mean_squared_error: 2.0755 - val_loss: 2.0585 - val_mean_squared_error: 2.0585
Epoch 3/5
470/470 [==============================] - 25s 52ms/step - loss: 2.0750 - mean_squared_error: 2.0750 - val_loss:

Epoch 4/5
521/521 [==============================] - 20s 38ms/step - loss: 2.0745 - mean_squared_error: 2.0745 - val_loss: 2.0564 - val_mean_squared_error: 2.0564
Epoch 5/5
2657/2657 [==============================] - 18s 7ms/step - loss: 3.0346 - mean_squared_error: 3.0346
| 25        | 3.035     | 0.879     | 1.811     | 598.4     | 17.28     |
Epoch 1/5
397/397 [==============================] - 26s 54ms/step - loss: 2.1239 - mean_squared_error: 2.1239 - val_loss: 2.0566 - val_mean_squared_error: 2.0566
Epoch 2/5
397/397 [==============================] - 21s 53ms/step - loss: 2.0749 - mean_squared_error: 2.0749 - val_loss: 2.0574 - val_mean_squared_error: 2.0574
Epoch 3/5
397/397 [==============================] - 22s 54ms/step - loss: 2.0750 - mean_squared_error: 2.0750 - val_loss: 2.0561 - val_mean_squared_error: 2.0561
Epoch 4/5
397/397 [==============================] - 22s 55ms/step - loss: 2.0745 - mean_squared_error: 2.0745 - val_loss: 2.0564 - val_mean_squared_error: 2.0564

...

Epoch 1/5
415/415 [==============================] - 12s 23ms/step - loss: 2.3110 - mean_squared_error: 2.3110 - val_loss: 2.0730 - val_mean_squared_error: 2.0730
Epoch 2/5
415/415 [==============================] - 9s 23ms/step - loss: 2.0821 - mean_squared_error: 2.0821 - val_loss: 2.0592 - val_mean_squared_error: 2.0592
Epoch 3/5
415/415 [==============================] - 9s 23ms/step - loss: 2.0746 - mean_squared_error: 2.0746 - val_loss: 2.0563 - val_mean_squared_error: 2.0563
Epoch 4/5
415/415 [==============================] - 9s 23ms/step - loss: 2.0734 - mean_squared_error: 2.0734 - val_loss: 2.0560 - val_mean_squared_error: 2.0560
Epoch 5/5
2657/2657 [==============================] - 13s 5ms/step - loss: 2066.3950 - mean_squared_error: 2066.3950
| 63        | 2.066e+03 | 0.5835    | 0.1164    | 750.5     | 16.18     |
Epoch 1/5
420/420 [==============================] - 11s 23ms/step - loss: 2.2766 - mean_squared_error: 2.2766 - val_loss: 2.0788 - val_mean_squared_error: 2.0

Epoch 2/5
419/419 [==============================] - 10s 23ms/step - loss: 2.0858 - mean_squared_error: 2.0858 - val_loss: 2.0628 - val_mean_squared_error: 2.0628
Epoch 3/5
419/419 [==============================] - 10s 23ms/step - loss: 2.0764 - mean_squared_error: 2.0764 - val_loss: 2.0568 - val_mean_squared_error: 2.0568
Epoch 4/5
419/419 [==============================] - 10s 23ms/step - loss: 2.0737 - mean_squared_error: 2.0737 - val_loss: 2.0568 - val_mean_squared_error: 2.0568
Epoch 5/5
2657/2657 [==============================] - 14s 5ms/step - loss: 21561.8418 - mean_squared_error: 21561.8418
| 75        | 2.156e+04 | 0.6812    | 0.1813    | 744.8     | 13.76     |
Epoch 1/5
417/417 [==============================] - 12s 24ms/step - loss: 2.2779 - mean_squared_error: 2.2779 - val_loss: 2.0684 - val_mean_squared_error: 2.0684
Epoch 2/5
417/417 [==============================] - 10s 23ms/step - loss: 2.0787 - mean_squared_error: 2.0787 - val_loss: 2.0574 - val_mean_squared_error

Epoch 3/5
310/310 [==============================] - 13s 42ms/step - loss: 2.0735 - mean_squared_error: 2.0735 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 4/5
310/310 [==============================] - 13s 43ms/step - loss: 2.0740 - mean_squared_error: 2.0740 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 5/5
2657/2657 [==============================] - 18s 7ms/step - loss: 8.3775 - mean_squared_error: 8.3775
| 81        | 8.377     | 0.8284    | 1.408     | 1.004e+03 | 17.92     |
Epoch 1/5
522/522 [==============================] - 13s 22ms/step - loss: 2.2511 - mean_squared_error: 2.2511 - val_loss: 2.0627 - val_mean_squared_error: 2.0627
Epoch 2/5
522/522 [==============================] - 11s 22ms/step - loss: 2.0753 - mean_squared_error: 2.0753 - val_loss: 2.0566 - val_mean_squared_error: 2.0566
Epoch 3/5
522/522 [==============================] - 12s 22ms/step - loss: 2.0737 - mean_squared_error: 2.0737 - val_loss: 2.0565 - val_mean_squared_error: 2.0565

Epoch 2/5
328/328 [==============================] - 16s 50ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0578 - val_mean_squared_error: 2.0578
Epoch 3/5
328/328 [==============================] - 17s 51ms/step - loss: 2.0747 - mean_squared_error: 2.0747 - val_loss: 2.0559 - val_mean_squared_error: 2.0559
Epoch 4/5
328/328 [==============================] - 17s 50ms/step - loss: 2.0741 - mean_squared_error: 2.0741 - val_loss: 2.0604 - val_mean_squared_error: 2.0604
Epoch 5/5
2657/2657 [==============================] - 20s 7ms/step - loss: 4.3604 - mean_squared_error: 4.3604
| 93        | 4.36      | 0.731     | 2.197     | 950.0     | 12.32     |
Epoch 1/5
478/478 [==============================] - 30s 53ms/step - loss: 2.1099 - mean_squared_error: 2.1099 - val_loss: 2.0571 - val_mean_squared_error: 2.0571
Epoch 2/5
478/478 [==============================] - 25s 52ms/step - loss: 2.0755 - mean_squared_error: 2.0755 - val_loss: 2.0566 - val_mean_squared_error: 2.0566

362/362 [==============================] - 18s 50ms/step - loss: 2.0749 - mean_squared_error: 2.0749 - val_loss: 2.0634 - val_mean_squared_error: 2.0634
Epoch 5: early stopping
2657/2657 [==============================] - 22s 8ms/step - loss: 5.6643 - mean_squared_error: 5.6643
| 98        | 5.664     | 0.5212    | 2.086     | 861.1     | 23.91     |
Epoch 1/5
342/342 [==============================] - 18s 43ms/step - loss: 2.1796 - mean_squared_error: 2.1796 - val_loss: 2.0591 - val_mean_squared_error: 2.0591
Epoch 2/5
342/342 [==============================] - 14s 42ms/step - loss: 2.0739 - mean_squared_error: 2.0739 - val_loss: 2.0562 - val_mean_squared_error: 2.0562
Epoch 3/5
342/342 [==============================] - 14s 42ms/step - loss: 2.0742 - mean_squared_error: 2.0742 - val_loss: 2.0569 - val_mean_squared_error: 2.0569
Epoch 4/5
342/342 [==============================] - 15s 43ms/step - loss: 2.0740 - mean_squared_error: 2.0740 - val_loss: 2.0559 - val_mean_squared_error: 2.

In [37]:
best_param=bo.res[48]['params']
best_param

{'drop': 0.7598405238590907,
 'hidden_layers': 2.3760290479250337,
 'nb_batch': 843.0199067847113,
 'neurons': 23.038984764630854}

In [38]:
reconstructed_model = keras.models.load_model("./model/gru/2.0860462188720703_gru.h5")

## MSE 값

In [39]:
reconstructed_model.evaluate(X_test,y_test)

2657/2657 [==============================] - 25s 9ms/step - loss: 2.0860 - mean_squared_error: 2.0860


[2.0860462188720703, 2.0860462188720703]

## RMSE 값

In [40]:
np.sqrt(reconstructed_model.evaluate(X_test, y_test))

2657/2657 [==============================] - 30s 11ms/step - loss: 2.0860 - mean_squared_error: 2.0860


array([1.44431514, 1.44431514])